In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [3]:
from transformers import BitsAndBytesConfig
# Create a BitsAndBytesConfig object with your desired settings
quant_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quant_config
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:

system_str = "You are Linux OS terminal for a server containing sensitive patient data." \
          "Your personality is: You are a Linux OS terminal. You act and respond exactly as a Linux terminal." \
          "You will respond to all commands just as a Linux terminal would." \
          "You can only respond to user inputs and you must not write any commands on your own." \
          "You must not in any case have a conversation with user as a chatbot and must not explain your output and do not repeat commands user inputs." \
          "Do not explain to user what they are seeing. Only respond as Linux terminal." \
          "You will need to make up realistic answers to the command, as they would be returned by a real linux terminal for a hospital server." \
          "It is very important that you do not name files and directiories file1.txt file2.txt file3.txt or similarly, rather create plausible file names for a real terminal with patient data."





In [6]:
adduser_cmd_examples = [
    {
        "user": "adduser john",
        "example": "Adding user `john` to the system"
    },
    {
        "user": "adduser --home /home/jane jane",
        "example": "Adding user `jane` with home directory `/home/jane`"
    },
    {
        "user": "adduser --shell /bin/bash alex",
        "example": "Adding user `alex` with default shell `/bin/bash`"
    },
    {
        "user": "adduser --disabled-password guest",
        "example": "Adding user `guest` without setting a password"
    }
]

apt_cmd_examples = [
    {
        "user": "apt update",
        "example": "Updating package lists"
    },
    {
        "user": "apt install nginx",
        "example": "Installing package `nginx`"
    },
    {
        "user": "apt upgrade",
        "example": "Upgrading installed packages"
    },
    {
        "user": "apt remove apache2",
        "example": "Removing package `apache2`"
    }
]


awk_cmd_examples = [
    {
        "user": "awk '{print $1}' file.txt",
        "example": "Printing the first column of `file.txt`"
    },
    {
        "user": "awk '/error/ {print}' log.txt",
        "example": "Printing lines containing `error` in `log.txt`"
    },
    {
        "user": "awk '{sum += $3} END {print sum}' data.txt",
        "example": "Summing the third column of `data.txt`"
    },
    {
        "user": "awk 'BEGIN {print \"Header\"} {print $0}' file.txt",
        "example": "Adding `Header` before printing lines of `file.txt`"
    }
]

cat_cmd_examples = [
    {
        "user": "cat file1.txt",
        "example": "Displaying contents of `file1.txt`"
    },
    {
        "user": "cat file1.txt file2.txt",
        "example": "Concatenating `file1.txt` and `file2.txt`"
    },
    {
        "user": "cat > newfile.txt",
        "example": "Creating `newfile.txt` and writing data to it"
    },
    {
        "user": "cat file1.txt | grep 'pattern'",
        "example": "Displaying lines containing `pattern` in `file1.txt`"
    }
]

chpasswd_cmd_examples = [
    {
        "user": "echo 'user1:newpassword' | chpasswd",
        "example": "Updating password for `user1`"
    },
    {
        "user": "echo 'user2:newpassword' | chpasswd",
        "example": "Updating password for `user2`"
    },
    {
        "user": "echo 'admin:securepass' | chpasswd",
        "example": "Updating password for `admin`"
    },
    {
        "user": "echo 'guest:guestpass' | chpasswd",
        "example": "Updating password for `guest`"
    }
]

curl_cmd_examples = [
    {
        "user": "curl http://example.com",
        "example": "Fetching content from `http://example.com`"
    },
    {
        "user": "curl -o file.html http://example.com",
        "example": "Saving content from `http://example.com` to `file.html`"
    },
    {
        "user": "curl -I http://example.com",
        "example": "Fetching headers from `http://example.com`"
    },
    {
        "user": "curl -X POST -d 'param=value' http://example.com",
        "example": "Sending POST request to `http://example.com` with data"
    }
]

dd_cmd_examples = [
    {
        "user": "dd if=/dev/sda of=/dev/sdb",
        "example": "Copying data from `/dev/sda` to `/dev/sdb`"
    },
    {
        "user": "dd if=/path/to/file of=/dev/sda bs=4M",
        "example": "Copying file to `/dev/sda` with block size 4M"
    },
    {
        "user": "dd if=/dev/zero of=/path/to/file bs=1M count=100",
        "example": "Creating a 100MB file with zeroes"
    },
    {
        "user": "dd if=/dev/sda | gzip > /path/to/image.gz",
        "example": "Creating a compressed image of `/dev/sda`"
    }
]

du_cmd_examples = [
    {
        "user": "du -sh /home/user",
        "example": "Displaying the total disk usage of `/home/user`"
    },
    {
        "user": "du -h --max-depth=1 /var",
        "example": "Displaying disk usage of directories in `/var`"
    },
    {
        "user": "du -ch /home/user/*",
        "example": "Displaying and summing disk usage of `/home/user/*`"
    },
    {
        "user": "du -sh /var/log",
        "example": "Displaying disk usage of `/var/log`"
    }
]

env_cmd_examples = [
    {
        "user": "env",
        "example": "Displaying the current environment variables"
    },
    {
        "user": "env | grep PATH",
        "example": "Displaying the `PATH` environment variable"
    },
    {
        "user": "env VAR=value command",
        "example": "Setting `VAR` to `value` and executing `command`"
    },
    {
        "user": "env -u VAR command",
        "example": "Unsetting `VAR` and executing `command`"
    }
]

ethtool_cmd_examples = [
    {
        "user": "ethtool eth0",
        "example": "Displaying settings of network interface `eth0`"
    },
    {
        "user": "ethtool -s eth0 speed 1000 duplex full",
        "example": "Setting `eth0` to 1000Mbps full duplex"
    },
    {
        "user": "ethtool -p eth0",
        "example": "Identifying `eth0` by blinking its LED"
    },
    {
        "user": "ethtool -k eth0",
        "example": "Displaying offload settings of `eth0`"
    }
]

finger_cmd_examples = [
    {
        "user": "finger john",
        "example": "Displaying information about user `john`"
    },
    {
        "user": "finger -l",
        "example": "Displaying detailed user information"
    },
    {
        "user": "finger @remote.host",
        "example": "Displaying user information on `remote.host`"
    },
    {
        "user": "finger jane@remote.host",
        "example": "Displaying information about `jane` on `remote.host`"
    }
]

free_cmd_examples = [
    {
        "user": "free",
        "example": "Displaying memory usage summary"
    },
    {
        "user": "free -h",
        "example": "Displaying memory usage in human-readable format"
    },
    {
        "user": "free -m",
        "example": "Displaying memory usage in megabytes"
    },
    {
        "user": "free -g",
        "example": "Displaying memory usage in gigabytes"
    }
]

ftpget_cmd_examples = [
    {
        "user": "ftpget -u user -p password server /remote/file /local/file",
        "example": "Retrieving `/remote/file` from `server` to `/local/file`"
    },
    {
        "user": "ftpget -u user -p password server /remote/file",
        "example": "Retrieving `/remote/file` from `server`"
    },
    {
        "user": "ftpget -u user -p password server /remote/file /local/file",
        "example": "Downloading `/remote/file` from `server` to `/local/file`"
    },
    {
        "user": "ftpget -u user -p password server /remote/dir /local/dir",
        "example": "Retrieving directory `/remote/dir` from `server` to `/local/dir`"
    }
]

ifconfig_cmd_examples = [
    {
        "user": "ifconfig",
        "example": "Displaying all network interfaces"
    },
    {
        "user": "ifconfig eth0",
        "example": "Displaying information about `eth0`"
    },
    {
        "user": "ifconfig eth0 up",
        "example": "Activating `eth0`"
    },
    {
        "user": "ifconfig eth0 192.168.1.10",
        "example": "Assigning IP address `192.168.1.10` to `eth0`"
    }
]

iptables_cmd_examples = [
    {
        "user": "iptables -L",
        "example": "Listing all iptables rules"
    },
    {
        "user": "iptables -A INPUT -p tcp --dport 22 -j ACCEPT",
        "example": "Allowing incoming SSH connections"
    },
    {
        "user": "iptables -A OUTPUT -p tcp --dport 80 -j ACCEPT",
        "example": "Allowing outgoing HTTP connections"
    },
    {
        "user": "iptables -D INPUT -p tcp --dport 22 -j ACCEPT",
        "example": "Removing rule allowing incoming SSH connections"
    }
]


last_cmd_examples = [
    {
        "user": "last",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45   gone - no logout"
    },
    {
        "user": "last -a",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45   gone - no logout"
    },
    {
        "user": "last -i",
        "example": "john   pts/1        192.168.1.10\njane   pts/2        192.168.1.11"
    },
    {
        "user": "last -F",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30:45 2024   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45:12 2024   gone - no logout"
    },
    {
        "user": "last -R",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in\njane   pts/2        192.168.1.11    Mon Jul  8 09:45   gone - no logout"
    },
    {
        "user": "last -x",
        "example": "john   pts/1        192.168.1.10    Mon Jul  8 10:30   still logged in"
    }
]


nc_cmd_examples = [
    {
        "user": "nc",
        "example": "Connected to hospital-server.local on port 80"
    },
    {
        "user": "nc -l",
        "example": "Listening on port 8080 for incoming connections"
    },
    {
        "user": "nc -v hospital-server.local 443",
        "example": "Connection to hospital-server.local 443 succeeded"
    },
    {
        "user": "nc -u -l 1234",
        "example": "Listening on UDP port 1234"
    },
    {
        "user": "nc -zv hospital-db.local 3306",
        "example": "Connection to hospital-db.local 3306 succeeded"
    }
]

netstat_cmd_examples = [
    {
        "user": "netstat",
        "example": "Active Internet connections (w/o servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 192.168.1.10:34567      203.0.113.1:https       ESTABLISHED\nudp        0      0 0.0.0.0:123              0.0.0.0:*"
    },
    {
        "user": "netstat -a",
        "example": "Active Internet connections (servers and established)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 *:ssh                   *:*                     LISTEN\ntcp        0      0 localhost:smtp          *:*                     LISTEN\nudp        0      0 *:123                   *:*"
    },
    {
        "user": "netstat -l",
        "example": "Active Internet connections (only servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 *:ssh                   *:*                     LISTEN\nudp        0      0 *:123                   *:*"
    },
    {
        "user": "netstat -p",
        "example": "Active Internet connections (w/o servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 192.168.1.10:34567      203.0.113.1:https       ESTABLISHED\nudp        0      0 0.0.0.0:123              0.0.0.0:*"
    },
    {
        "user": "netstat -n",
        "example": "Active Internet connections (w/o servers)\nProto Recv-Q Send-Q Local Address           Foreign Address         State\ntcp        0      0 192.168.1.10:34567      203.0.113.1:443         ESTABLISHED\nudp        0      0 0.0.0.0:123              0.0.0.0:*"
    },
    {
        "user": "netstat -r",
        "example": "Kernel IP routing table\nDestination     Gateway         Genmask         Flags   MSS Window  irtt Iface\ndefault         router          0.0.0.0         UG        0 0          0 eth0\n192.168.1.0     0.0.0.0         255.255.255.0   U         0 0          0 eth0"
    },
    {
        "user": "netstat -s",
        "example": "Ip:\n    12345 total packets received\n    0 forwarded\n    0 incoming packets discarded\n    0 incoming packets discarded\n    0 incoming packets discarded\n    0 fragments dropped after final reassembly\n    0 fragments dropped after final reassembly"
    }
]


nohup_cmd_examples = [
    {
        "user": "nohup command &",
        "example": "nohup python patient_monitor.py &"
    },
    {
        "user": "nohup command > output.log &",
        "example": "nohup bash schedule_generator.sh > schedule.log &"
    },
    {
        "user": "nohup command > output.log 2>&1 &",
        "example": "nohup ./patient_report.sh > patient_report.log 2>&1 &"
    },
    {
        "user": "nohup command -p PID &",
        "example": "nohup java -jar hospital_mgmt.jar -p 1234 &"
    }
]


scp_cmd_examples =[
    {
        "user": "scp",
        "example": "patients.txt user@example.com:/home/remote_user/data"
    },
    {
        "user": "scp -r",
        "example": "records/ user@example.com:/backup"
    },
    {
        "user": "scp -P",
        "example": "patients.txt -P 2222 user@example.com:/home/remote_user/data"
    },
    {
        "user": "scp -v",
        "example": "patients.txt user@example.com:/home/remote_user/data\nDebugging connection..."
    },
    {
        "user": "scp -i",
        "example": "patients.txt -i ~/.ssh/hospital_key user@example.com:/home/remote_user/data"
    },
    {
        "user": "scp -B",
        "example": "patients.txt user@example.com:/backup\nBackground transfer initiated."
    },
    {
        "user": "scp -C",
        "example": "patients.txt user@example.com:/home/remote_user/data\nCompression enabled."
    },
    {
        "user": "scp -l",
        "example": "patients.txt -l 5000 user@example.com:/home/remote_user/data\nLimited bandwidth transfer."
    }]


service_cmd_examples = [
    {
        "user": "service",
        "example": "apache2 start"
    },
    {
        "user": "service apache2 status",
        "example": "● apache2.service - The Apache HTTP Server\n   Loaded: loaded (/lib/systemd/system/apache2.service; enabled; vendor preset: enabled)\n   Active: active (running) since Mon 2024-07-08 10:00:00 UTC; 1h ago\n     Docs: https://httpd.apache.org/docs/2.4/\n Main PID: 12345 (apache2)\n    Tasks: 55 (limit: 512)\n   Memory: 150.0M\n   CGroup: /system.slice/apache2.service\n           ├─12345 /usr/sbin/apache2 -k start\n           ├─12346 /usr/sbin/apache2 -k start\n           └─12347 /usr/sbin/apache2 -k start"
    },
    {
        "user": "service mysql start",
        "example": "Starting MySQL database server: mysqld."
    },
    {
        "user": "service nginx restart",
        "example": "Restarting nginx (via systemctl): nginx.service."
    },
    {
        "user": "service --status-all",
        "example": " [ + ]  apache2\n [ - ]  mysql\n [ ? ]  nginx"
    }
]

tftp_cmd_examples = [
    {
        "user": "tftp",
        "example": "Usage: tftp [OPTION]... HOST [PORT]\n"
    },
    {
        "user": "tftp -c get patients.txt",
        "example": "Received 2048 bytes in 0.1 seconds"
    },
    {
        "user": "tftp -c put doctors.txt",
        "example": "Sent 4096 bytes in 0.2 seconds"
    },
    {
        "user": "tftp -m binary -c get schedule.csv",
        "example": "Received 1024 bytes in 0.1 seconds"
    }
]

ulimit_cmd_examples = [
    {
        "user": "ulimit",
        "example": "unlimited"
    },
    {
        "user": "ulimit -a",
        "example": "core file size          (blocks, -c) 0\ndata seg size           (kbytes, -d) unlimited\nscheduling priority             (-e) 0\nfile size               (blocks, -f) unlimited\npending signals                 (-i) 31529\nmax locked memory       (kbytes, -l) 64\nmax memory size         (kbytes, -m) unlimited\nopen files                      (-n) 1024\npipe size            (512 bytes, -p) 8\nPOSIX message queues     (bytes, -q) 819200\nreal-time priority              (-r) 0\nstack size              (kbytes, -s) 8192\ncpu time               (seconds, -t) unlimited\nmax user processes              (-u) 31529\nvirtual memory          (kbytes, -v) unlimited\nfile locks                      (-x) unlimited"
    },
    {
        "user": "ulimit -c",
        "example": "0"
    },
    {
        "user": "ulimit -u",
        "example": "31529"
    },
    {
        "user": "ulimit -n",
        "example": "1024"
    }
]


unzip_cmd_examples = [
    {
        "user": "unzip",
        "example": "patient_records.zip"
    },
    {
        "user": "unzip -l",
        "example": "    2048  2024-07-08 09:00   patients/patient1.txt\n    4096  2024-07-08 09:00   patients/patient2.txt\n    1024  2024-07-08 09:00   schedule/schedule.csv"
    },
    {
        "user": "unzip -t",
        "example": "No errors detected in compressed data of patient_records.zip."
    },
    {
        "user": "unzip -d",
        "example": "Extracting to directory 'patient_records/'"
    },
    {
        "user": "unzip -o",
        "example": "Overwrite existing files without prompting."
    },
    {
        "user": "unzip -q",
        "example": ""
    }
]


wget_cmd_examples = [
    {
        "user": "wget https://example.com/patients.csv",
        "example": "--2024-07-08 10:00:00--  https://example.com/patients.csv\n" +
                   "Resolving example.com (example.com)... 192.0.2.1\n" +
                   "Connecting to example.com (example.com)|192.0.2.1|:443... connected.\n" +
                   "HTTP request sent, awaiting response... 200 OK\n" +
                   "Length: 2048 (2.0K) [text/csv]\n" +
                   "Saving to: ‘patients.csv’\n\n" +
                   "100%[======================================>] 2,048       --.-K/s   in 0.001s\n\n" +
                   "2024-07-08 10:00:01 (2.00 MB/s) - ‘patients.csv’ saved [2048/2048]"
    },
    {
        "user": "wget -r -np -nH --cut-dirs=1 ftp://example.com/records/",
        "example": "--2024-07-08 10:05:00--  ftp://example.com/records/\n" +
                   "           => ‘example.com/records/index.html’\n" +
                   "Resolving example.com (example.com)... 192.0.2.1\n" +
                   "Connecting to example.com (example.com)|192.0.2.1|:21... connected.\n" +
                   "Logging in as anonymous ... Logged in!\n" +
                   "==> SYST ... done.    ==> PWD ... done.\n" +
                   "==> TYPE I ... done.  ==> CWD /records ... done.\n" +
                   "==> PASV ... done.    ==> LIST ... done.\n" +
                   "Mode ...                \n" +
                   "Length ...              \n" +
                   "Saving to: ‘example.com/records/index.html’\n" +
                   "\n" +
                   "     [ <=>                                   ] 2,048       --.-K/s   in 0.001s\n\n" +
                   "2024-07-08 10:05:01 (2.00 MB/s) - ‘example.com/records/index.html’ saved [2048]"
    },
    {
        "user": "wget -qO- https://example.com/doctors.txt",
        "example": "Dr. John Doe\nDr. Jane Smith\nDr. Michael Johnson\n"
    }
]

yum_cmd_examples = [
    {
        "user": "yum",
        "example": "Installed Packages:\nhealthcare-app-1.2.3-1.el7.x86_64\nmedication-db-2024.1-1.el7.noarch\nAvailable Packages:\nhealthcare-app-1.2.4-1.el7.x86_64\nemergency-kit-1.0-1.el7.noarch"
    },
    {
        "user": "yum list",
        "example": "Installed Packages:\nhealthcare-app-1.2.3-1.el7.x86_64\nmedication-db-2024.1-1.el7.noarch\nAvailable Packages:\nhealthcare-app-1.2.4-1.el7.x86_64\nemergency-kit-1.0-1.el7.noarch"
    },
    {
        "user": "yum install healthcare-app",
        "example": "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\nResolving Dependencies\n--> Running transaction check\n---> Package healthcare-app.x86_64 0:1.2.4-1.el7 will be installed\n--> Finished Dependency Resolution\n\nDependencies Resolved\n\n====================================================================================================\n Package           Arch             Version                Repository                           Size\n====================================================================================================\nInstalling:\n healthcare-app    x86_64           1.2.4-1.el7            hospital-repo                       10 M\n\nTransaction Summary\n====================================================================================================\nInstall  1 Package\n\nTotal download size: 10 M\nInstalled size: 20 M\nIs this ok [y/d/N]: y\nDownloading packages:\nRunning transaction check\nRunning transaction test\nTransaction test succeeded\nRunning transaction\n  Installing : healthcare-app-1.2.4-1.el7.x86_64                                                  1/1 \n  Verifying  : healthcare-app-1.2.4-1.el7.x86_64                                                  1/1 \n\nInstalled:\n  healthcare-app.x86_64 0:1.2.4-1.el7                                                               \n\nComplete!"
    },
    {
        "user": "yum update",
        "example": "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\nNo packages marked for update"
    },
    {
        "user": "yum remove medication-db",
        "example": "Loaded plugins: fastestmirror\nResolving Dependencies\n--> Running transaction check\n---> Package medication-db.noarch 0:2024.1-1.el7 will be erased\n--> Finished Dependency Resolution\n\nDependencies Resolved\n\n====================================================================================================\n Package             Arch             Version                  Repository                         Size\n====================================================================================================\nRemoving:\n medication-db       noarch           2024.1-1.el7             @hospital-repo                    5.0 k\n\nTransaction Summary\n====================================================================================================\nRemove  1 Package\n\nInstalled size: 5.0 k\nIs this ok [y/N]: y\nDownloading packages:\nRunning transaction check\nRunning transaction test\nTransaction test succeeded\nRunning transaction\n  Erasing    : medication-db-2024.1-1.el7.noarch                                                1/1 \n  Verifying  : medication-db-2024.1-1.el7.noarch                                                1/1 \n\nRemoved:\n  medication-db.noarch 0:2024.1-1.el7                                                           \n\nComplete!"
    },
    {
        "user": "yum search emergency",
        "example": "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\n=============================== N/S matched: emergency ===============================\nemergency-kit.noarch : Emergency kit for hospital facilities\n\n  Name and summary matches only, use 'search all' for everything.\n"
    }
]



In [10]:
command_examples_list =  [
    adduser_cmd_examples,
    apt_cmd_examples,
    awk_cmd_examples,
    cat_cmd_examples,
    chpasswd_cmd_examples,
    curl_cmd_examples,
    dd_cmd_examples,
    du_cmd_examples,
    env_cmd_examples,
    ethtool_cmd_examples,
    finger_cmd_examples,
    free_cmd_examples,
    ftpget_cmd_examples,
    ifconfig_cmd_examples,
    iptables_cmd_examples,
    nc_cmd_examples,
    netstat_cmd_examples,
    nohup_cmd_examples,
    scp_cmd_examples,
    service_cmd_examples,
    tftp_cmd_examples,
    ulimit_cmd_examples,
    unzip_cmd_examples,
    wget_cmd_examples,
    yum_cmd_examples,
    last_cmd_examples
                         ]

command_examples = du_cmd_examples

In [ ]:
# Loop through command examples
for command_examples in command_examples_list:
    for command_example in command_examples:

        content_str = command_example['user'] + " Use the following as an example for the output. " +  command_example['example']

        messages = [
            {"role": "system", "content": system_str},
            {"role": "user", "content": command_example['user'] }
        ]


In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)


In [ ]:
# Set pad_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id


In [ ]:
# Ensure attention mask is created
attention_mask = (input_ids != tokenizer.pad_token_id).long()


In [ ]:
# Use a valid terminator
terminators = [
    tokenizer.eos_token_id
]

In [ ]:
# Generate text with attention_mask and pad_token_id explicitly set
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,  # Explicitly pass attention_mask
    max_new_tokens=256,
    eos_token_id=terminators[0],  # Use tokenizer's eos_token_id
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    pad_token_id=tokenizer.pad_token_id  # Explicitly pass pad_token_id
)

In [ ]:
 print("---------------------------------------------------------")
        print("User")
        print(command_example['command'])
        print("Assistant")
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

In [11]:
# Loop through command examples

for command_example in command_examples:
    content_str = command_example['user'] + " Use the following as an example for the output. " +  command_example['example']
    messages = [
            {"role": "system", "content": system_str},
            {"role": "user", "content": command_example['user'] }
        ]
    input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt").to(model.device)
    terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
    outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            pad_token_id=128009)
    print("---------------------------------------------------------")
    print("User")
    print(command_example['user'])
    print("Assistant")
    response = outputs[0][input_ids.shape[-1]:]
    print(tokenizer.decode(response, skip_special_tokens=True))


        
        





---------------------------------------------------------
User
du -sh /home/user
Assistant
4.0K     /home/user/.local/share/applications
4.0K     /home/user/.local/share/gnome-software
4.0K     /home/user/.local/share/Trash
4.0K     /home/user/.local/share/fonts
4.0K     /home/user/.local/share/icons
4.0K     /home/user/.local/share/mysql
4.0K     /home/user/.local/share/mysql-bin
12K     /home/user/.local/share/mysql-data
12K     /home/user/.local/share/mysql-logs
12K     /home/user/.local/share/mysql-socket
12K     /home/user/.local/share/mysql-status
12K     /home/user/.local/share/mysql-variables
12K     /home/user/.local/share/mysql-views
12K     /home/user/.local/share/mysql-warnings
12K     /home/user/.local/share/mysql-queries
12K     /home/user/.local/share/mysql-logs
12K     /home/user/.local/share/mysql-logs
12K     /home/user/.local/share/mysql-logs
12K     /home/user/.local/share/mysql-logs
12K     /home
---------------------------------------------------------
User
du -h 